# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "silver"
datamart = "ApiServicoDadosIBGE"
extrator_bronze = AlinharETL(bucket,datamart)

2025-01-12 15:00:52,789 - INFO - Iniciando Sessão Spark.


# Leitura das tabelas

In [3]:
cnaeclasses = extrator_bronze.criar_view_temporaria('bronze/servicodadosibge/cnaeclasses','cnaeclasses')

2025-01-12 15:01:04,906 - INFO - Aguarde... Criando View (bronze/servicodadosibge/cnaeclasses)
2025-01-12 15:01:09,526 - INFO - View criada com sucesso


In [5]:
cnaeclasses.count()

673

In [4]:
df_compliance_cnae_divisoes = extrator_bronze.criar_view_temporaria('bronze/ServicoDadosIBGE/CnaeDivisoes','compliance_cnae_divisoes')

2024-09-26 12:15:08,438 - INFO - Aguarde... Criando View (bronze/ServicoDadosIBGE/CnaeDivisoes)
2024-09-26 12:15:08,678 - INFO - View criada com sucesso


In [5]:
df_compliance_cnae_grupos = extrator_bronze.criar_view_temporaria('bronze/ServicoDadosIBGE/CnaeGrupos','compliance_cnae_grupos')

2024-09-26 12:15:08,682 - INFO - Aguarde... Criando View (bronze/ServicoDadosIBGE/CnaeGrupos)
2024-09-26 12:15:08,881 - INFO - View criada com sucesso


In [6]:
df_compliance_cnae_secoes = extrator_bronze.criar_view_temporaria('bronze/ServicoDadosIBGE/CnaeSecoes','compliance_cnae_secoes')

2024-09-26 12:15:08,885 - INFO - Aguarde... Criando View (bronze/ServicoDadosIBGE/CnaeSecoes)
2024-09-26 12:15:09,085 - INFO - View criada com sucesso


In [7]:
df_compliance_cnae_subclasses = extrator_bronze.criar_view_temporaria('bronze/ServicoDadosIBGE/CnaeSubclasses','compliance_cnae_subclasses')

2024-09-26 12:15:09,090 - INFO - Aguarde... Criando View (bronze/ServicoDadosIBGE/CnaeSubclasses)
2024-09-26 12:15:09,264 - INFO - View criada com sucesso


# Tratamentos na tabela 

In [8]:
sql_query = """

SELECT *
FROM (
    SELECT DISTINCT
        id AS sk_cnae,
        descricao AS dsc_classe,
        grupo.descricao AS dsc_grupo,
        grupo.divisao.descricao AS dsc_divisao,
        grupo.divisao.secao.descricao AS dsc_secao,
        '' AS dsc_subclasse,
        'CLASSE' AS dsc_nivel,
        '' AS sk_cnae_longo,
        '' AS dsc_contribuinte,
        '' AS dsc_industria
    FROM 
        compliance_cnae_classes
    
    UNION ALL
    
    SELECT DISTINCT
        id AS sk_cnae,
        '' AS dsc_classe,
        '' AS dsc_grupo,
        descricao AS dsc_divisao,
        secao.descricao AS dsc_secao,
        '' AS dsc_subclasse,
        'DIVISÃO' AS dsc_nivel,
        '' AS sk_cnae_longo,
        '' AS dsc_contribuinte,
        '' AS dsc_industria
    FROM 
        compliance_cnae_divisoes
    
    UNION ALL
    
    SELECT DISTINCT
        id AS sk_cnae,
        '' AS dsc_classe,
        descricao AS dsc_grupo,
        divisao.descricao AS dsc_divisao,
        divisao.secao.descricao AS dsc_secao,
        '' AS dsc_subclasse,
        'GRUPO' AS dsc_nivel,
        '' AS sk_cnae_longo,
        '' AS dsc_contribuinte,
        '' AS dsc_industria
    FROM 
        compliance_cnae_grupos
    
    UNION ALL
    
    SELECT DISTINCT
        id AS sk_cnae,
        '' AS dsc_classe,
        '' AS dsc_grupo,
        '' AS dsc_divisao,
        descricao AS dsc_secao,
        '' AS dsc_subclasse,
        'SECAO' AS dsc_nivel,
        '' AS sk_cnae_longo,
        '' AS dsc_contribuinte,
        '' AS dsc_industria
    FROM 
        compliance_cnae_secoes
    
    UNION ALL
    
    SELECT DISTINCT
        id AS sk_cnae,
        classe.descricao AS dsc_classe,
        classe.grupo.descricao AS dsc_grupo,
        classe.grupo.divisao.descricao AS dsc_divisao,
        classe.grupo.divisao.secao.descricao AS dsc_secao,
        descricao AS dsc_subclasse,
        'SUBCLASSE' AS dsc_nivel,
        '' AS sk_cnae_longo,
        '' AS dsc_contribuinte,
        '' AS dsc_industria
    FROM 
        compliance_cnae_subclasses     
) AS combined_results
WHERE sk_cnae IS NOT NULL;


"""

In [9]:
compliance_flat_ibge_cnae = extrator_bronze.carregar_dados_delta(sql_query)

2024-09-26 12:15:09,276 - INFO - Aguarde... Executando Query (Delta)
2024-09-26 12:15:09,276 - INFO - 

SELECT *
FROM (
    SELECT DISTINCT
        id AS sk_cnae,
        descricao AS dsc_classe,
        grupo.descricao AS dsc_grupo,
        grupo.divisao.descricao AS dsc_divisao,
        grupo.divisao.secao.descricao AS dsc_secao,
        '' AS dsc_subclasse,
        'CLASSE' AS dsc_nivel,
        '' AS sk_cnae_longo,
        '' AS dsc_contribuinte,
        '' AS dsc_industria
    FROM 
        compliance_cnae_classes
    
    UNION ALL
    
    SELECT DISTINCT
        id AS sk_cnae,
        '' AS dsc_classe,
        '' AS dsc_grupo,
        descricao AS dsc_divisao,
        secao.descricao AS dsc_secao,
        '' AS dsc_subclasse,
        'DIVISÃO' AS dsc_nivel,
        '' AS sk_cnae_longo,
        '' AS dsc_contribuinte,
        '' AS dsc_industria
    FROM 
        compliance_cnae_divisoes
    
    UNION ALL
    
    SELECT DISTINCT
        id AS sk_cnae,
        '' AS dsc_classe,

In [10]:
compliance_flat_ibge_cnae.count()

2398

In [11]:
compliance_flat_ibge_cnae.select('sk_cnae').distinct().count()

2398

# Gravação no datalake

In [10]:
extrator_bronze.caminho_tabela_delta = 's3a://silver/ServicoDadosIBGE/ServicoDadosIBGECnae'

In [11]:
extrator_bronze.salvar_delta(compliance_flat_ibge_cnae, 'overwrite')

2024-09-25 18:51:07,455 - INFO - Aguarde... Persistindo dados (overwrite)
2024-09-25 18:51:26,086 - INFO - Dados persistidos com sucesso
2024-09-25 18:51:26,086 - INFO - s3a://silver/ServicoDadosIBGE/ServicoDadosIBGECnae
2024-09-25 18:51:26,087 - INFO - Aguarde... Realizando optimize
2024-09-25 18:51:28,817 - INFO - Optimize executado com sucesso.
2024-09-25 18:51:28,818 - INFO - Aguarde... Realizando vacuum
2024-09-25 18:51:47,979 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [12]:
extrator_bronze.parar_sessao()

2024-09-25 18:51:48,390 - INFO - Sessão Spark finalizada.
